In [1]:
## If you run for first time uncomment these
# !pip install elasticsearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from scipy import spatial
from elasticsearch import Elasticsearch

In [2]:
#Parameters
no_of_recepies = 500 # Total = 124473
ingredient_list = 'Filterd_ingredient' # ingredients (otherwise)

Reading Pickel File for Filtered Data and creating dictionary of recipes

In [4]:
%%time
# Gagan's Code
import pandas as pd
rec_df = pd.read_pickle("Cleaned_recipes.P")
rec_l = []
temp_dict = rec_df.to_dict(orient='index')
for ind in temp_dict:
    rec_l.append(temp_dict[ind])
es = Elasticsearch()
for i in range(0,len(rec_l[:no_of_recepies])): #i took only first 120 recipes because time and size
    res = es.index(index='recipes', doc_type='recipe', id=i, body=rec_l[i])

Wall time: 1min 27s


In [3]:
# %%time
# recipes = []
# with open('recipes_raw_nosource_ar.json') as json_file: #make sure the recipe json is in same dir
#     data = json.load(json_file)
#     for p in data:
#         recipes.append(data[p])
# es = Elasticsearch()
# for i in range(0,len(recipes[:500])): #i took only first 120 recipes because time and size
#     res = es.index(index='recipes', doc_type='recipe', id=i, body=recipes[i])

Wall time: 1min 26s


In [5]:
def giveResults(query, last, wish,title=True,instructions=True):
    res = es.search(index="recipes", body={"query": {"bool": { "must": [{"match":{ingredient_list:query} }]}},"size":no_of_recepies})
    presults = [] #personalized
    mscores = [] # new scores for personalized results
    results = [] #unpersonalized
    for hit in res['hits']['hits']:
        if title:
            results.append((hit['_source']['title'],hit['_score']))
        if instructions: 
            results.append((hit['_source']['instructions'],hit['_score']))
        ingr = ''
        for s in hit['_source'][ingredient_list]:
            ingr += s
        ldocuments = (ingr,last)
        wdocuments = (ingr,wish)
        tfidf_vectorizer=TfidfVectorizer()
        tfidf_last=tfidf_vectorizer.fit_transform(ldocuments)
        tfidf_wish=tfidf_vectorizer.fit_transform(wdocuments)
        cs = -100 * cosine_similarity(tfidf_last[0:1],tfidf_last)
        ds = 100 * cosine_similarity(tfidf_wish[0:1],tfidf_wish)
        mscores.append((hit['_source'],hit['_score']+cs[0][1]+ds[0][1]))
    mscores.sort(key=lambda t: t[1],reverse=True)
    for t in mscores:
        if title:
            presults.append((t[0]['title'],t[1]))
        if instructions:
            presults.append((t[0]['instructions'],t[1]))
    return presults, results

In [6]:
def getInput():
    query = input("Enter whatever ingredients you have in your pantry seperated by whitespaces: ")
    last = input("Enter at least 5 ingredients you ate last week seperated by whitespace: ")
    wish = input("List anything you would like to eat. You can also just skip this.")
    return query,last,wish

In [7]:
def giveRecipes(title = True, instructions = False):
    query,last,wish = getInput()
    print(query)
    personalized, standard = giveResults(query,last,wish,title,instructions)
    return personalized, standard

In [ ]:
p, s = giveRecipes()

In [8]:
s

[('Spanish Rice II', 6.0677805),
 ('Stuffed Green Peppers I', 5.41187),
 ('Best Spanish Rice', 4.258619),
 ('Cabbage Roll Casserole', 3.9156854),
 ('Creamy Rice Pudding', 3.8133276),
 ('Mexican Rice II', 3.623867),
 ('Chinese Chicken Fried Rice II', 3.623867),
 ('Garlic Mashed Potatoes Secret Recipe', 3.338746),
 ('Red Skinned Potato Salad', 3.2491264),
 ('Creamy Chicken and Wild Rice Soup', 3.223481),
 ('Honey Roasted Red Potatoes', 3.1641927),
 ('Double Tomato Bruschetta', 3.1327112),
 ('Stuffed Peppers', 3.087051),
 ('Easy Irish Colcannon', 3.0835857),
 ('Creamy Au Gratin Potatoes', 3.0835857),
 ('Slow Cooker Mashed Potatoes', 3.0835857),
 ('Gourmet Mushroom Risotto', 3.023077),
 ('Cabbage Rolls II', 3.023077),
 ('Ultimate Twice Baked Potatoes', 3.0069838),
 ("Marie's Easy Slow Cooker Pot Roast", 3.0069838),
 ('Sweet Potato Casserole II', 3.0069838),
 ('Guacamole', 2.9626343),
 ('Cheddar Baked Chicken', 2.9617004),
 ('Salsa Chicken Rice Casserole', 2.9617004),
 ('Delicious Ham and P

In [8]:
s

[('Spanish Rice II', 6.0677805),
 ('Stuffed Green Peppers I', 5.41187),
 ('Best Spanish Rice', 4.258619),
 ('Cabbage Roll Casserole', 3.9156854),
 ('Creamy Rice Pudding', 3.8133276),
 ('Mexican Rice II', 3.623867),
 ('Chinese Chicken Fried Rice II', 3.623867),
 ('Garlic Mashed Potatoes Secret Recipe', 3.338746),
 ('Red Skinned Potato Salad', 3.2491264),
 ('Creamy Chicken and Wild Rice Soup', 3.223481),
 ('Honey Roasted Red Potatoes', 3.1641927),
 ('Double Tomato Bruschetta', 3.1327112),
 ('Stuffed Peppers', 3.087051),
 ('Easy Irish Colcannon', 3.0835857),
 ('Creamy Au Gratin Potatoes', 3.0835857),
 ('Slow Cooker Mashed Potatoes', 3.0835857),
 ('Gourmet Mushroom Risotto', 3.023077),
 ('Cabbage Rolls II', 3.023077),
 ('Ultimate Twice Baked Potatoes', 3.0069838),
 ("Marie's Easy Slow Cooker Pot Roast", 3.0069838),
 ('Sweet Potato Casserole II', 3.0069838),
 ('Guacamole', 2.9626343),
 ('Cheddar Baked Chicken', 2.9617004),
 ('Salsa Chicken Rice Casserole', 2.9617004),
 ('Delicious Ham and P

In [8]:
s

[('Spanish Rice II', 6.244555),
 ('Stuffed Green Peppers I', 5.410444),
 ('Best Spanish Rice', 4.3023167),
 ('Mexican Rice II', 3.878198),
 ('Cabbage Roll Casserole', 3.8155096),
 ('Creamy Rice Pudding', 3.7548156),
 ('Chinese Chicken Fried Rice II', 3.6960223),
 ('Absolutely Ultimate Potato Soup', 3.685146),
 ('Stuffed Peppers', 3.4276688),
 ('Creamy Chicken and Wild Rice Soup', 3.3309305),
 ('Garlic Mashed Potatoes Secret Recipe', 3.327356),
 ('Honey Roasted Red Potatoes', 3.327356),
 ('Double Tomato Bruschetta', 3.3041506),
 ('Red Skinned Potato Salad', 3.2993317),
 ('Salsa Chicken Rice Casserole', 3.239503),
 ('Easy Irish Colcannon', 3.2180204),
 ("Marie's Easy Slow Cooker Pot Roast", 3.1660032),
 ('Cheddar Baked Chicken', 3.15296),
 ('Cabbage Rolls II', 3.15296),
 ('Gourmet Mushroom Risotto', 3.070921),
 ('Ultimate Twice Baked Potatoes', 3.0668561),
 ('Cheesy Ham and Hash Brown Casserole', 3.0195756),
 ('Slow Cooker Mashed Potatoes', 3.0195756),
 ('Creamy Au Gratin Potatoes', 2.97

In [9]:
p

[('Creamy Chicken and Wild Rice Soup', 19.652324595038873),
 ('Six Can Chicken Tortilla Soup', 18.539156539183942),
 ('Chicken Milano', 18.484925742129988),
 ('Salsa Chicken Rice Casserole', 12.621816885451427),
 ('Slow Cooker Mashed Potatoes', 12.548827468786433),
 ('Mulligatawny Soup I', 10.89727929809862),
 ('Best Spanish Rice', 9.960047933054627),
 ('Gourmet Mushroom Risotto', 9.950007822376403),
 ("Colleen's Slow Cooker Jambalaya", 9.526454833145886),
 ('Hash Brown Casserole II', 9.2106865065138),
 ('Cheddar Baked Chicken', 8.649104637687095),
 ('Chicken Tortilla Soup I', 8.41177064213639),
 ('Slow-Cooker Chicken Tortilla Soup', 8.12829423378715),
 ('Mexican Rice II', 7.567003936339676),
 ('Chinese Chicken Fried Rice II', 7.020362277940922),
 ('Chicken Breasts Pierre', 6.726066818400868),
 ('Butternut Squash Soup II', 6.2611522779409245),
 ('Super-Delicious Zuppa Toscana', 6.070256024392644),
 ("Catherine's Spicy Chicken Soup", 6.038616867530795),
 ('Absolutely Ultimate Potato Sou

In [9]:
p

[('Creamy Chicken and Wild Rice Soup', 19.759774095038875),
 ('Chicken Milano', 18.705345342129988),
 ('Six Can Chicken Tortilla Soup', 18.54071863918394),
 ('Salsa Chicken Rice Casserole', 13.777594728236883),
 ('Slow Cooker Mashed Potatoes', 12.484817368786432),
 ('Best Spanish Rice', 11.771476795699208),
 ('Mulligatawny Soup I', 11.692903202762986),
 ("Colleen's Slow Cooker Jambalaya", 10.294784677118317),
 ('Gourmet Mushroom Risotto', 9.997851822376404),
 ('Hash Brown Casserole II', 9.265376006513801),
 ('Chicken Tortilla Soup I', 9.122703772295578),
 ('Mexican Rice II', 9.043906702608808),
 ('Cheddar Baked Chicken', 8.840364237687096),
 ('Slow-Cooker Chicken Tortilla Soup', 8.799541579547697),
 ('Chinese Chicken Fried Rice II', 8.145602798697075),
 ('Absolutely Ultimate Potato Soup', 7.89690667996085),
 ('Super-Delicious Zuppa Toscana', 7.496758784174968),
 ('Butternut Squash Soup II', 7.423310798697075),
 ("Catherine's Spicy Chicken Soup", 7.275179392189504),
 ('Chicken Breasts P